In [1]:
import os
import rioxarray as rxr
import numpy as np
import pdemtools as pdt
from glob import glob
from math import isnan

index_fpath = 'R:/datasets/ArcticDEM/ArcticDEM_Strip_Index_s2s041_gpqt/ArcticDEM_Strip_Index_s2s041.parquet'
bm_fpath = 'R:/datasets/Bed_Machine/BedMachineGreenland-v5.nc'

In [5]:
# SELECT EDITABLE PARAMETER SECTION #

# # # KOGE BUGT NORTH EDITABLE PARAMETER SECTION # # # 
region = "kogebugtnorth" # the name of your study area 
dataset = "arcticdem" # "arcticdem" or "rema"
xmin, ymin, xmax, ymax = 195237, -2725348, 210876, -2710389 # the bounding box of your study area
dates = "20160101/20240101" # the date range of interest
baseline_max_hours = 24 
min_aoi_frac = 0.25 # the minimum fraction of the bounding box that must be covered by the strip
months=[6, 7, 8, 9] # the months of interest
outdir = f"R:/KOGE_BUGT/QGIS/ArcticDEM/Koge_Bugt_North/{region}_data" # the output directory
if not os.path.exists(outdir):
    os.mkdir(outdir)
# # # KOGE BUGT NORTH EDITABLE PARAMETER SECTION # # # 

# # # # KOGE BUGT CENTRAL EDITABLE PARAMETER SECTION # # # 
region = "kogebugtcentral" # the name of your study area 
dataset = "arcticdem" # "arcticdem" or "rema"
xmin, ymin, xmax, ymax = 169805, -2731604, 192925, -2713925 # the bounding box of your study area
dates = "20160101/20240101" # the date range of interest
baseline_max_hours = 24 
min_aoi_frac = 0.25 # the minimum fraction of the bounding box that must be covered by the strip
months=[6, 7, 8, 9] # the months of interest
outdir = f"R:/KOGE_BUGT/QGIS/ArcticDEM/Koge_Bugt_Central/{region}_data" # the output directory
if not os.path.exists(outdir):
    os.mkdir(outdir)
# # # KOGE BUGT CENTRAL EDITABLE PARAMETER SECTION # # # 

# # # KOGE BUGT SOUTH EDITABLE PARAMETER SECTION # # #
region = "kogebugtsouth" # the name of your study area 
dataset = "arcticdem" # "arcticdem" or "rema"
xmin, ymin, xmax, ymax = 170213, -2752820, 189933, -2738948 # the bounding box of your study area
dates = "20160101/20240101" # the date range of interest
baseline_max_hours = 24 
min_aoi_frac = 0.25 # the minimum fraction of the bounding box that must be covered by the strip
months=[6, 7, 8, 9] # the months of interest
outdir = f"R:/KOGE_BUGT/QGIS/ArcticDEM/Koge_Bugt_South/{region}_data" # the output directory
if not os.path.exists(outdir):
    os.mkdir(outdir)
# # # KOGE BUGT SOUTH EDITABLE PARAMETER SECTION # # #

In [ ]:
# # # INSERT EDITABLE PARAMETER SECTION # # # 

region = "kogebugtsouth" # the name of your study area 
dataset = "arcticdem" # "arcticdem" or "rema"
xmin, ymin, xmax, ymax = 170213, -2752820, 189933, -2738948 # the bounding box of your study area
dates = "20160101/20240101" # the date range of interest
baseline_max_hours = 24 
min_aoi_frac = 0.25 # the minimum fraction of the bounding box that must be covered by the strip
months=[6, 7, 8, 9] # the months of interest
outdir = f"R:/KOGE_BUGT/QGIS/ArcticDEM/Koge_Bugt_South/{region}_data" # the output directory
if not os.path.exists(outdir):
    os.mkdir(outdir)

# # # END OF EDITABLE PARAMETER SECTION # # #

bounds = (xmin, ymin, xmax, ymax)
print(f"Downloading data for {region}:")
reference_dem_fpath = os.path.join(outdir, f"{region}_arcticdem_mosaic_2m.tif")
if not os.path.exists(reference_dem_fpath):
    print("\nDownloading reference DEM...")
    reference_dem = pdt.load.mosaic(dataset=dataset, resolution=2, bounds=bounds, version="v4.1")
    reference_dem.rio.to_raster(reference_dem_fpath, compress="ZSTD", predictor=3, zlevel=1)
else:
    print("\nLoading reference DEM...")
    reference_dem = pdt.load.from_fpath(
        os.path.join(outdir, f"{region}_arcticdem_mosaic_2m.tif"), bounds=bounds)
reference_dem = reference_dem.squeeze()
bedrock_mask = pdt.data.bedrock_mask_from_bedmachine(bm_fpath, reference_dem)
print("\nSearching for DEM strips...")
gdf = pdt.search(index_fpath, bounds, dates=dates, months= months, baseline_max_hours=baseline_max_hours, sensors=["WV03", "WV02", "WV01"], accuracy=2, min_aoi_frac=min_aoi_frac)
gdf = gdf.sort_values("acqdate1")
n_strips = len(gdf)
print(f"{n_strips} strips found")
i = 1

print("\nDownloading DEM strips...")
for _, row in gdf.iterrows():
    date = row.acqdate1.date()
    date_str = date.strftime("%Y%m%d")
    dem_id = row.dem_id
    out_fname = os.path.join(outdir, f"{date_str}_{dem_id}")

    if len(glob(f'{out_fname}*')) == 0:
        dem = pdt.load.from_search(row, bounds=bounds, bitmask=True)
        dem.compute() 
        dem = dem.rio.pad_box(*bounds, constant_values=np.nan)
        dem = dem.pdt.coregister(reference_dem, bedrock_mask, max_horiz_offset=50, return_stats=True)
        rmse = dem[-1]
        dem = dem[0]
        if isnan(rmse) == True:
            out_fpath = out_fname + '.tif'
        else:
            out_fpath = out_fname + '_coreg.tif'
        dem.rio.to_raster(out_fpath, compress="ZSTD", predictor=3, zlevel=1)
        del dem
    i += 1
print("Download finished!")